In [16]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, f1_score
from sklearn import set_config
from sklearn import datasets
from sklearn import ensemble, neighbors,linear_model,svm
import eli5
set_config(display='diagram')



#datasets.load*?

In [17]:
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

In [18]:
# df = sklearn_to_df(datasets.load_breast_cancer())
# target = 'target'
# drop_columns = []

In [22]:
df = pd.read_csv('input/geyser.csv')
target = 'waiting'
drop_columns = []

In [23]:
df.head()

,eruptions,waiting
0,3.600,79
1,1.800,54
2,3.333,74
3,2.283,62
4,4.533,85


In [24]:
df = df.drop(drop_columns, axis = 1)

if df[target].dtypes == np.object:    
    numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = df.select_dtypes(include=['object']).drop(target, axis=1).columns
else:
    numeric_features = df.select_dtypes(include=['int64', 'float64']).drop(target, axis=1).columns
    categorical_features = df.select_dtypes(include=['object']).columns
    
X = df.drop((target),axis = 'columns')
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state =1)

In [25]:
#ProfileReport(df)

In [26]:
# train_values = pd.read_csv('train_values.csv')
# train_labels = pd.read_csv('train_labels.csv')
# train_data = train_values.merge(train_labels, left_on='building_id', right_on='building_id')

# train_data = train_data.drop('building_id', axis=1)
# numeric_features = train_data.select_dtypes(include=['int64', 'float64']).drop(['damage_grade'], axis=1).columns
# categorical_features = train_data.select_dtypes(include=['object']).columns
# X = train_data.drop('damage_grade', axis=1)
# y = train_data['damage_grade']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn import ensemble, neighbors,linear_model,svm
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('feature_selection',SelectFromModel(svm.LinearSVC(penalty = 'l1',dual = False))),
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree = 2,interaction_only = True)),
    ])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='most_frequent')),
    ('one_hot', OneHotEncoder()),
    ])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model',  ensemble.RandomForestRegressor(max_depth = 3))])
    
#model = pipe.fit(X_train, y_train)
pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('feature_selection',
                                                                   SelectFromModel(estimator=LinearSVC(dual=False,
                                                                                                       penalty='l1'))),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('poly',
                                                                   PolynomialFeatures(interaction_only=True))]),
                                                  Index(['eruptions'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='most_frequent',
                                                                                 strategy='constant')),
                                                                  ('one_hot',
                                                                   OneHotEncoder())]),
                                                  Index([], dtype='object'))])),
                ('model', RandomForestRegressor(max_depth=3))])

In [28]:
model_list = [#regressors
                linear_model.LinearRegression(),linear_model.Lasso(),linear_model.Ridge(),
              
              #ensembling - regressors
                ensemble.RandomForestRegressor(max_depth = 3),ensemble.AdaBoostRegressor(),ensemble.GradientBoostingRegressor(),
    
              #classifiers
                linear_model.LogisticRegression(max_iter = 500), neighbors.KNeighborsClassifier(),svm.SVC(),             
    
              #ensembling - classifiers
                ensemble.RandomForestClassifier(n_estimators=500,max_depth = 3),ensemble.AdaBoostClassifier(learning_rate=0.5),ensemble.GradientBoostingClassifier()

              ]

In [29]:
rejected_models = []
results = []
print('{:30s}Train  /  Test   /  Cross Validation'.format('Model'))
print('-----------------------------------------------------------------')
for model in model_list:
    pipe                 = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])
    
    try:        
        pipe.fit(X_train,y_train)
        
        model_name       = type(model).__name__
        train_results    = np.abs(pipe.score(X_train,y_train))
        test_results     = np.abs(pipe.score(X_test,y_test))
        cross_val_scores = np.abs(cross_val_score(pipe, X, y, cv=5)) 
        cval_score       = cross_val_scores.mean()
        
        results.append([model_name, train_results, test_results,cval_score])
        print('{:30s}{:.4f} / {:.4f}  /  {:.4f} '.format(model_name, train_results, test_results,cval_score))

    except:
        rejected_models.append(type(model).__name__)

df_results = pd.DataFrame(results, columns=["Model", "Train Results","Test Results",'Test Cross Validation Results'])

Model                         Train  /  Test   /  Cross Validation
-----------------------------------------------------------------
LinearRegression              0.8017 / 0.8279  /  0.8053 
Lasso                         0.7960 / 0.8103  /  0.8014 
Ridge                         0.8017 / 0.8270  /  0.8054 
RandomForestRegressor         0.8472 / 0.8187  /  0.8118 
AdaBoostRegressor             0.8465 / 0.8053  /  0.8042 
GradientBoostingRegressor     0.8977 / 0.7673  /  0.7800 
LogisticRegression            0.1053 / 0.0244  /  0.0809 
KNeighborsClassifier          0.2368 / 0.0122  /  0.0441 
SVC                           0.1053 / 0.0244  /  0.0882 
RandomForestClassifier        0.2579 / 0.0366  /  0.0772 
AdaBoostClassifier            0.1105 / 0.0366  /  0.0554 
GradientBoostingClassifier    0.5684 / 0.0366  /  0.0477 


In [30]:
if rejected_models == []:
    print('No models rejected')
else:    
    print('Rejected Models list: ')
    for rej_models in rejected_models:
        print('  - ',rej_models)

No models rejected


In [31]:
def highlight_range(s):
    '''
    highlight the maximum in a Series green.
    '''
    is_max = s == s.max()
    #threshold = 0.95
    #val = s > threshold
    return ['background-color: lightgreen' if v else '' for v in is_max]

df_results.sort_values(by = 'Test Cross Validation Results',ascending = False)\
.style.apply(highlight_range,subset=pd.IndexSlice[:, ['Train Results', 'Test Results', 'Test Cross Validation Results']])

,Model,Train Results,Test Results,Test Cross Validation Results
3,RandomForestRegressor,0.847231,0.818674,0.811785
2,Ridge,0.801672,0.827037,0.805395
0,LinearRegression,0.801694,0.827854,0.805309
4,AdaBoostRegressor,0.846544,0.805346,0.804236
1,Lasso,0.796031,0.810259,0.801353
5,GradientBoostingRegressor,0.897703,0.767269,0.780023
8,SVC,0.105263,0.024390,0.088215
6,LogisticRegression,0.105263,0.024390,0.080875
9,RandomForestClassifier,0.257895,0.036585,0.077172
10,AdaBoostClassifier,0.110526,0.036585,0.055354


### Feature Selection

In [14]:
# try:
#     pipe = Pipeline(steps=[('preprocessor', preprocessor),('model',  ensemble.RandomForestRegressor(max_depth = 3))])
#     pipe.fit(X_train,y_train)
# except:
#     pipe = Pipeline(steps=[('preprocessor', preprocessor),('model',  ensemble.RandomForestClassifier(max_depth = 3))])
#     pipe.fit(X_train,y_train)

# try:
#     onehot_columns = list(pipe.named_steps['preprocessor'].named_transformers_['cat'].named_steps['one_hot']
#                           .get_feature_names(input_features=categorical_features))
# except:
#     onehot_columns = []

# numeric_features_list = list(numeric_features)
# numeric_features_list.extend(onehot_columns)
# eli5.explain_weights(pipe.named_steps['model'], top=10, feature_names=numeric_features_list)

### TroubleShoot

In [15]:
# rejected_models = []
# results = []
# print('{:30s}Train  /  Test   /  Cross Validation'.format('Model'))
# print('-----------------------------------------------------------')
# for model in model_list:
#     pipe                 = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])   
#     pipe.fit(X_train,y_train)
#     model_name       = type(model).__name__
#     train_results    = np.abs(pipe.score(X_train,y_train))
#     test_results     = np.abs(pipe.score(X_test,y_test))
#     cross_val_scores = np.abs(cross_val_score(pipe, X, y, cv=5)) 
#     cval_score       = cross_val_scores.mean()

#     print('{:30s}{:.4f} / {:.4f}  /  {:.4f} '.format(model_name, train_results, test_results,cval_score))